# Scrapping de letterboxd

Pasos a seguir:
1. extraer todos los usuarios que tengan listas armadas y armar un csv de usuarios: id_usuario, link_usuario
2. para cada usuario, extraer los links a sus las listas y armar un csv de listas: id_usuario, id_lista, link_lista
3. para cada lista, exportar a un csv con las interacciones: id_lista, id_pelicula, posicion, link_pelicula
4. para cada link_pelicula, obtener los datos de las peliculas

al final hay que tener 3 archivos:
interacciones (listas-peliculas)
peliculas
listas

In [2]:
import requests
import lxml
from lxml import etree
from lxml import html
import xml.etree.ElementTree as ET
import pandas as pd
import os
from pathlib import Path
import csv


 Paso 1: extraer todos los usuarios que tengan listas armadas y armar un csv de usuarios: id_usuario, link_usuario


In [102]:
path = "data/users.csv"
file = "users.csv"

for number in range(1,1000):
    r = requests.get(f"https://letterboxd.com/members/popular/this/week/page/{number}/")
    tree = lxml.html.fromstring(r.text)
    table = tree.xpath('//*[@id="content"]/div/div/section/table/tbody')
    
    # si la tabla no está vacía de filas
    if len(table[0].findall("tr")) != 0:
        # guardo las variables
        user_link = [element.attrib["href"] for element in table[0].findall(".//*[@class='title-3']/a")]
        user_lists_link = [element.attrib["href"] for element in table[0].findall(".//*[@class='has-icon icon-16 icon-list']")]
        user_watched_num = [int(element.text.replace(",", "")) for element in table[0].findall(".//*[@class='has-icon icon-16 icon-watched']")]
        user_lists_num = [int(element.text.replace(",", "")) for element in table[0].findall(".//*[@class='has-icon icon-16 icon-list']")]
        user_likes_num = [int(element.text.replace(",", "")) for element in table[0].findall(".//*[@class='has-icon icon-16 icon-liked']")]
        
        #armo el dataframe
        df_to_append = pd.DataFrame()
        df_to_append = pd.DataFrame(
            {'user_link': user_link,
            'user_lists_link': user_lists_link,
            'user_watched_num': user_watched_num,
            'user_lists_num': user_lists_num,
            'user_likes_num': user_likes_num
            })
        #header del csv
        header = ["user_link", "user_lists_link", "user_watched_num", "user_lists_num", "user_likes_num"]
        
        # si no existe el csv, se crea con el header
        if os.path.isfile(path)==False:
            with open(file, 'a') as f:
                writer = csv.writer(f)
                writer.writerow(header)
                df_to_append.to_csv(f, header=False, index=False)
                f.flush()
        
        # si existe el csv, se appendea el dataframe
        else:
            with open(file, 'a') as f:
                df_to_append.to_csv(f, header=False, index=False)
                f.flush()
    
    else:
        break




2. para cada usuario, extraer los links a sus las listas y armar un csv de listas: id_usuario, id_lista, link_lista

In [3]:
#para cada usuario
#loopear sobre páginas hasta que no se devuelva ninguna película

r = requests.get("https://letterboxd.com/angienoexiste/lists/page/2")
tree = lxml.html.fromstring(r.text)
table = tree.xpath('//*[@id="content"]/div/div/section/section')

In [11]:
list_link = [element.attrib["href"] for element in table[0].findall(".//*[@class='list-link']")]
list_id = [element.attrib['data-film-list-id'] for element in table[0].findall(".//*[@class='list -overlapped -summary ']")]
user_id = [element.attrib['data-person'] for element in table[0].findall(".//*[@class='list -overlapped -summary ']")]
list_extension = [int(element.text.replace('\xa0films', '')) for element in table[0].findall(".//*[@class='value']")]

['/angienoexiste/list/michael-haneke-ranked/',
 '/angienoexiste/list/alfred-hitchcock-ranked/',
 '/angienoexiste/list/christopher-nolan-ranked/',
 '/angienoexiste/list/denis-villeneuve-ranked/',
 '/angienoexiste/list/oriol-paulo-ranked/',
 '/angienoexiste/list/jean-marc-vallee-ranked/',
 '/angienoexiste/list/martin-scorsese-ranked/']

In [131]:
#importar datos de usuarios
users = pd.read_csv("data/users_1.csv")
users

,user_link,user_lists_link,user_watched_num,user_lists_num,user_likes_num
0,/filmento/,/filmento/lists/,161,0,3
1,/dawsonejoyce/,/dawsonejoyce/lists/,7027,1481,45607
2,/denielemein/,/denielemein/lists/,1923,24,6435
3,/cusack/,/cusack/lists/,2776,50,22167
4,/sarab3lla/,/sarab3lla/lists/,1510,2,57726
...,...,...,...,...,...
2494,/iamgarvpahal/,/iamgarvpahal/lists/,910,2,1161
2495,/harley_memer69/,/harley_memer69/lists/,1566,104,7024
2496,/midnightgospel/,/midnightgospel/lists/,1775,31,5968
2497,/gravesends/,/gravesends/lists/,1629,71,4190


In [132]:
#tengo que loopear esta lista de usuarios
users_list_links = "https://letterboxd.com" + users.user_lists_link + "page/"

In [134]:
path = "data/lists.csv"
file = "lists.csv"

#para cada usuario
for user in users_list_links:
    #tengo que ver todas sus páginas
    for number in range(1, 10000):
        r = requests.get(f"{user}{number}")
        tree = lxml.html.fromstring(r.text)
        table = tree.xpath('//*[@id="content"]/div/div/section/section')
        
        if len(table[0].findall(".//*[@class='list -overlapped -summary ']")) != 0:
            
            list_link = [element.attrib["href"] for element in table[0].findall(".//*[@class='list-link']")]
            list_id = [element.attrib['data-film-list-id'] for element in table[0].findall(".//*[@class='list -overlapped -summary ']")]
            user_id = [element.attrib['data-person'] for element in table[0].findall(".//*[@class='list -overlapped -summary ']")]
            list_extension = [element.text for element in table[0].findall(".//*[@class='value']")]
        
            #armo el dataframe
            df_to_append = pd.DataFrame()
            df_to_append = pd.DataFrame(
                {'user_id': user_id,
                'list_id': list_id,
                'list_link': list_link,
                'list_extension': list_extension
                })

            #header del csv
            header = ["user_id", "list_id", "list_link", "list_extension"]

            # si no existe el csv, se crea con el header
            if os.path.isfile(path)==False:
                with open(path, 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow(header)
                    df_to_append.to_csv(f, header=False, index=False)
                    f.flush()

            # si existe el csv, se appendea el dataframe
            else:
                with open(path, 'a') as f:
                    df_to_append.to_csv(f, header=False, index=False)
                    f.flush()

        else:
            break    


KeyboardInterrupt: 

3. para cada lista, exportar a un csv con las interacciones: id_lista, id_pelicula, posicion, link_pelicula

In [206]:
#ejemplo de lista

#https://letterboxd.com/emmaertzgf/list/movies-about-teenage-years-childhood/
#js-list-entries poster-list -p125 -grid film-list

r = requests.get("https://letterboxd.com/emmaertzgf/list/movies-about-teenage-years-childhood/")
tree = lxml.html.fromstring(r.text)
table = tree.xpath('//*[@id="content"]/div/div/section/ul')
table[0].attrib

{'class': 'js-list-entries poster-list -p125 -grid film-list'}

In [205]:
#link a los posters
# id_pelicula
#link_pelicula

movie_id = [element.attrib["data-film-id"] for element in table[0].findall(".//*[@class='poster-container']/div")]

In [202]:
# https://stackoverflow.com/questions/73803684/trying-to-scrape-posters-from-letterboxd-python
movie_link = [element.attrib["data-target-link"] for element in table[0].findall(".//*[@class='poster-container']/div")]

['/film/call-me-by-your-name/',
 '/film/but-im-a-cheerleader/',
 '/film/summer-of-85/',
 '/film/the-florida-project/',
 '/film/the-perks-of-being-a-wallflower/',
 '/film/wonder-2017/',
 '/film/tomboy-2011/',
 '/film/aftersun/',
 '/film/stand-by-me/',
 '/film/speak/',
 '/film/instant-family/',
 '/film/mid90s/',
 '/film/the-new-kid/',
 '/film/the-breakfast-club/',
 '/film/beautiful-boy-2018/',
 '/film/sydney-white/',
 '/film/10-things-i-hate-about-you/',
 '/film/clueless/',
 '/film/billy-elliot/',
 '/film/the-bling-ring/',
 '/film/lady-bird/',
 '/film/boyhood/',
 '/film/booksmart/',
 '/film/juno/',
 '/film/its-the-great-pumpkin-charlie-brown/',
 '/film/my-first-summer/',
 '/film/fantastic-mr-fox/',
 '/film/my-girl/',
 '/film/the-virgin-suicides/',
 '/film/the-edge-of-seventeen/',
 '/film/eighth-grade/',
 '/film/my-life-as-a-zucchini/',
 '/film/little-miss-sunshine/',
 '/film/water-lilies/',
 '/film/bring-it-on/',
 '/film/superbad/',
 '/film/good-old-daze/',
 '/film/palo-alto/',
 '/film/m

In [208]:
import requests
from bs4 import BeautifulSoup as bs
import json

#url = 'https://letterboxd.com/film/jojo-rabbit/'

#r = requests.get(url)
soup = bs(r.text)
print(soup)


<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="en" class="ie6 lte9 lte8 lte7 lte6 no-js"> <![endif]--><!--[if IE 7 ]>    <html lang="en" class="ie7 lte9 lte8 lte7 no-js"> <![endif]--><!--[if IE 8 ]>    <html lang="en" class="ie8 lte9 lte8 no-js"> <![endif]--><!--[if IE 9 ]>    <html lang="en" class="ie9 lte9 no-js"> <![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class="no-mobile no-js" id="html" lang="en"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=1024" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="A list of 49 films compiled on Letterboxd, including Call Me by Your Name (2017), But I'm a Cheerleader (1999), Summer of 85 (2020), The Florida Project (2017) and The Perks of Being a Wallflower (2012). About this list: ou alors avec des ados / enfants en perso principal" name="description"/>
<meta content="letterboxd:list" property="og:type"/>
<meta content="https://letterboxd.com/emmaertzgf/list/movies-about-te

In [212]:
script_w_data = soup.select_one('script[type="application/ld+json"]')
#json_obj = json.loads(script_w_data.text.split(' */')[1].split('/* ]]>')[0])
#print(json_obj['image'])
script_w_data